In [1]:
import torch
from pyhealth.datasets import eICUDataset

In [ ]:
dataset = eICUDataset(
        root="D:/Downloads/eicu-collaborative-research-database-2.0",
        tables=["diagnosis", "medication", "lab", "treatment", "physicalExam"],
    )
dataset.stat()
dataset.info()

Parsing lab:  15%|█████████                                                    | 28905/195730 [01:12<03:50, 724.33it/s]